## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
sys.path.append("../")


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Margate,GB,51.3813,1.3862,42.96,66,29,5.99,scattered clouds
1,1,Anzhero-Sudzhensk,RU,56.0810,86.0285,33.46,97,100,14.79,snow
2,2,Ushuaia,AR,-54.8000,-68.3000,42.42,87,75,5.75,broken clouds
3,3,Mar Del Plata,AR,-38.0023,-57.5575,69.10,90,20,11.99,few clouds
4,4,Hithadhoo,MV,-0.6000,73.0833,82.35,70,29,10.04,scattered clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Mar Del Plata,AR,-38.0023,-57.5575,69.10,90,20,11.99,few clouds
4,4,Hithadhoo,MV,-0.6000,73.0833,82.35,70,29,10.04,scattered clouds
5,5,Rikitea,PF,-23.1203,-134.9692,79.29,77,100,18.19,overcast clouds
6,6,Saint-Philippe,RE,-21.3585,55.7679,78.39,87,100,16.71,light rain
7,7,Mayo,US,38.8876,-76.5119,77.04,58,0,9.22,clear sky
11,11,Cape Town,ZA,-33.9258,18.4232,71.28,86,75,16.11,broken clouds
12,12,Kahului,US,20.8947,-156.4700,84.22,51,20,9.22,few clouds
13,13,Yaan,NG,7.3833,8.5667,82.22,62,100,12.26,overcast clouds
14,14,Arraial Do Cabo,BR,-22.9661,-42.0278,76.86,83,0,14.97,clear sky
18,18,Tautira,PF,-17.7333,-149.1500,87.82,77,33,15.28,light rain


In [20]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.count()
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()


City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Mar Del Plata,AR,69.10,few clouds,-38.0023,-57.5575,
4,Hithadhoo,MV,82.35,scattered clouds,-0.6000,73.0833,
5,Rikitea,PF,79.29,overcast clouds,-23.1203,-134.9692,
6,Saint-Philippe,RE,78.39,light rain,-21.3585,55.7679,
7,Mayo,US,77.04,clear sky,38.8876,-76.5119,
11,Cape Town,ZA,71.28,broken clouds,-33.9258,18.4232,
12,Kahului,US,84.22,few clouds,20.8947,-156.4700,
13,Yaan,NG,82.22,overcast clouds,7.3833,8.5667,
14,Arraial Do Cabo,BR,76.86,clear sky,-22.9661,-42.0278,
18,Tautira,PF,87.82,light rain,-17.7333,-149.1500,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [42]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
print(clean_hotel_df.shape)
clean_hotel_df.head()


(355, 7)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Mar Del Plata,AR,69.10,few clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
4,Hithadhoo,MV,82.35,scattered clouds,-0.6000,73.0833,Scoop Guest House
5,Rikitea,PF,79.29,overcast clouds,-23.1203,-134.9692,People ThankYou
6,Saint-Philippe,RE,78.39,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
7,Mayo,US,77.04,clear sky,38.8876,-76.5119,Resorts Resource Group International


In [44]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [48]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [49]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))